# Конспект: Загрузка, очистка и преобразование данных в Python

Этот ноутбук охватывает полный цикл предварительной обработки данных на примере датасета `SalesTarget.csv`.

Столбцы:
- `row_id`: числовой индекс
- `Category`: категория товара (Office Supplies, Furniture, Technology)
- `Order Date`: дата заказа
- `Segment`: сегмент клиента (Consumer, Home Office)
- `Sales Target`: целевой показатель продаж

Цель: научиться загружать, чистить и преобразовывать данные с использованием `pandas` и `numpy`.

In [1]:
# Импорт библиотек
import pandas as pd
import numpy as np

## 1. Загрузка данных

### О чём пойдёт речь:
- `pd.read_csv()` — загрузка из CSV
- Обработка пробелов в названиях столбцов
- Указание пропущенных значений
- Преобразование типов при загрузке

In [2]:
# Загрузка данных
df = pd.read_csv('SalesTarget.csv',
                 na_values=['', 'N/A', 'NULL'],  # Какие значения считать NaN
                 dtype={'row_id': 'Int64'}       # Позволяем Int64 с NaN
                )

# Переименуем столбец с пробелом для удобства
df = df.rename(columns={'Sales Target': 'Sales_Target'})

print(f"Размер данных: {df.shape}")
print(f"Столбцы: {list(df.columns)}")
df.head()

Размер данных: (4603, 5)
Столбцы: ['row_id', 'Category', 'Order Date', 'Segment', 'Sales_Target']


,row_id,Category,Order Date,Segment,Sales_Target
0,0,Office Supplies,2017-01-04,Consumer,15
1,1,Office Supplies,2017-01-05,Home Office,300
2,2,Office Supplies,2017-01-06,Consumer,21
3,3,Furniture,2017-01-07,Home Office,2316
4,4,Technology,2017-01-07,Home Office,1068


## 2. Очистка данных

### О чём пойдёт речь:
- Проверка и обработка пропущенных значений
- Удаление и заполнение NaN
- Работа с дубликатами
- Приведение типов данных
- Очистка строк: пробелы, регистр
- Удаление столбцов/строк
- Замена значений
- Обработка выбросов (IQR)
- Валидация данных

### Проверка пропущенных значений

In [3]:
# Проверка NaN
print("Количество пропусков:")
print(df.isna().sum())

print(f"\nВсего пропусков: {df.isna().sum().sum()}")

Количество пропусков:
row_id          0
Category        0
Order Date      0
Segment         0
Sales_Target    0
dtype: int64

Всего пропусков: 0


### Удаление строк с пропусками

In [4]:
# Удаление строк, где есть NaN в ключевых столбцах
df_clean = df.dropna(subset=['Category', 'Order Date', 'Segment', 'Sales_Target']).copy()

print(f"После удаления пропусков: {df_clean.shape}")

После удаления пропусков: (4603, 5)


### Работа с дубликатами

In [5]:
# Поиск полных дубликатов
duplicates = df_clean.duplicated().sum()
print(f"Полных дубликатов: {duplicates}")

# Удаление дубликатов
df_clean = df_clean.drop_duplicates().copy()
print(f"После удаления дубликатов: {df_clean.shape}")

Полных дубликатов: 0
После удаления дубликатов: (4603, 5)


### Приведение типов данных

In [6]:
# Преобразование даты
df_clean['Order Date'] = pd.to_datetime(df_clean['Order Date'], errors='coerce')

# Преобразование Sales_Target в числовой
df_clean['Sales_Target'] = pd.to_numeric(df_clean['Sales_Target'], errors='coerce')

# Приведение к строке с поддержкой NaN
df_clean['Category'] = df_clean['Category'].astype('string')
df_clean['Segment'] = df_clean['Segment'].astype('string')

print("Типы данных после преобразования:")
print(df_clean.dtypes)

Типы данных после преобразования:
row_id                   Int64
Category        string[python]
Order Date      datetime64[ns]
Segment         string[python]
Sales_Target             int64
dtype: object


### Очистка строк: пробелы и регистр

In [7]:
# Удаление пробелов и приведение к стандартному регистру
df_clean['Category'] = df_clean['Category'].str.strip().str.title()
df_clean['Segment'] = df_clean['Segment'].str.strip().str.title()

print("Уникальные значения после очистки:")
print("Category:", df_clean['Category'].unique())
print("Segment:", df_clean['Segment'].unique())

Уникальные значения после очистки:
Category: <StringArray>
['Office Supplies', 'Furniture', 'Technology']
Length: 3, dtype: string
Segment: <StringArray>
['Consumer', 'Home Office', 'Corporate']
Length: 3, dtype: string


### Удаление столбцов

In [8]:
# Удаление row_id, если не нужен
if 'row_id' in df_clean.columns:
    df_clean = df_clean.drop(columns=['row_id'])

print(f"Осталось столбцов: {df_clean.shape[1]}")

Осталось столбцов: 4


### Замена значений

In [9]:
# Замена для улучшения читаемости
category_map = {
    'Office Supplies': 'Office & Supplies',
    'Furniture': 'Furniture & Decor',
    'Technology': 'Electronics & Tech'
}

segment_map = {
    'Home Office': 'Remote Worker',
    'Consumer': 'Individual Buyer'
}

df_clean['Category'] = df_clean['Category'].replace(category_map)
df_clean['Segment'] = df_clean['Segment'].replace(segment_map)

print("После замены:\n", df_clean[['Category', 'Segment']].head(3))

После замены:
             Category           Segment
0  Office & Supplies  Individual Buyer
1  Office & Supplies     Remote Worker
2  Office & Supplies  Individual Buyer


### Обработка выбросов (IQR-метод)

In [10]:
# Функция удаления выбросов
def remove_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return df[(df[col] >= lower) & (df[col] <= upper)]

df_clean = remove_outliers_iqr(df_clean, 'Sales_Target')
print(f"После удаления выбросов: {df_clean.shape[0]} строк")

После удаления выбросов: 4203 строк


### Валидация данных

In [11]:
# Проверка диапазона Sales_Target
invalid_sales = df_clean[(df_clean['Sales_Target'] <= 0)]
if len(invalid_sales) > 0:
    print(f"Найдено {len(invalid_sales)} строк с некорректным Sales_Target")
    df_clean = df_clean[df_clean['Sales_Target'] > 0]

# Проверка дат
df_clean = df_clean[df_clean['Order Date'].dt.year.between(2010, 2025)]

print(f"Финальный размер: {df_clean.shape}")

Финальный размер: (4203, 4)


## 3. Преобразование данных

### О чём пойдёт речь:
- Группировка и агрегация
- Сводные таблицы
- Создание новых столбцов
- Условные вычисления
- Фильтрация и сортировка
- Изменение индекса

### Группировка и агрегация

In [12]:
grouped = df_clean.groupby(['Category', 'Segment']).agg({
    'Sales_Target': ['sum', 'mean', 'count'],
    'Order Date': 'min'
}).round(2)

grouped.columns = ['_'.join(col).strip() for col in grouped.columns]
grouped = grouped.reset_index()

print("Сводка по категориям и сегментам:")
grouped

Сводка по категориям и сегментам:


,Category,Segment,Sales_Target_sum,Sales_Target_mean,Sales_Target_count,Order Date_min
0,Electronics & Tech,Corporate,105807,321.60,329,2017-01-14
1,Electronics & Tech,Individual Buyer,172688,362.03,477,2017-01-10
2,Electronics & Tech,Remote Worker,70497,329.43,214,2017-01-07
3,Furniture & Decor,Corporate,130572,346.34,377,2017-01-11
4,Furniture & Decor,Individual Buyer,192250,362.05,531,2017-01-08
5,Furniture & Decor,Remote Worker,73056,294.58,248,2017-01-27
6,Office & Supplies,Corporate,141005,207.06,681,2017-01-07
7,Office & Supplies,Individual Buyer,209787,241.97,867,2017-01-04
8,Office & Supplies,Remote Worker,80713,168.50,479,2017-01-05


### Сводные таблицы

In [14]:
# Добавим год
df_clean['Order_Year'] = df_clean['Order Date'].dt.year

pivot = pd.pivot_table(df_clean,
                       values='Sales_Target',
                       index='Category',
                       columns='Order_Year',
                       aggfunc='sum',
                       fill_value=0)

print("Сводная таблица по годам:")
pivot

Сводная таблица по годам:


Order_Year,2017,2018,2019,2020
Category,,,,
Electronics & Tech,69225,84048,87358,108361
Furniture & Decor,87638,84420,106921,116899
Office & Supplies,82007,89928,117503,142067


### Создание новых столбцов

In [15]:
df_clean['Sales_Category'] = df_clean['Sales_Target'].apply(
    lambda x: 'High' if x > 500 else 'Low'
)

print("Новый столбец 'Sales_Category':")
print(df_clean[['Sales_Target', 'Sales_Category']].head(3))

Новый столбец 'Sales_Category':
   Sales_Target Sales_Category
0            15            Low
1           300            Low
2            21            Low


### Условные вычисления

In [16]:
df_clean['Bonus'] = np.where(
    df_clean['Sales_Target'] > 200,
    df_clean['Sales_Target'] * 0.1,
    df_clean['Sales_Target'] * 0.05
)

print("Столбец 'Bonus' добавлен:")
print(df_clean[['Sales_Target', 'Bonus']].head(3))

Столбец 'Bonus' добавлен:
   Sales_Target  Bonus
0            15   0.75
1           300  30.00
2            21   1.05


### Фильтрация и сортировка

In [17]:
filtered = df_clean[
    (df_clean['Category'] == 'Office & Supplies') &
    (df_clean['Sales_Target'] > 100)
]

sorted_df = filtered.sort_values(by='Sales_Target', ascending=False)

print("Отфильтровано и отсортировано:")
sorted_df.head(3)

Отфильтровано и отсортировано:


,Category,Order Date,Segment,Sales_Target,Order_Year,Sales_Category,Bonus
3016,Office & Supplies,2019-11-25,Individual Buyer,1287,2019,High,128.7
3067,Office & Supplies,2019-12-04,Individual Buyer,1287,2019,High,128.7
3073,Office & Supplies,2019-12-06,Individual Buyer,1283,2019,High,128.3


### Изменение индекса

In [18]:
df_indexed = df_clean.set_index('Order Date')
df_reset = df_indexed.reset_index()

print("Индекс изменён и восстановлен.")

Индекс изменён и восстановлен.


## Заключение

Этот конспект охватывает полный цикл предварительной обработки данных:
- Загрузка с обработкой пробелов и типов
- Глубокая очистка: NaN, дубли, строки, выбросы
- Преобразование: группировка, сводные таблицы, условия, новые столбцы

Все операции выполнены безопасно, без предупреждений, с использованием современных практик `pandas`.

📌 **Совет**: Всегда используйте `.copy()` после фильтрации и `astype('string')` для строк с NaN.